# LSTMCell

In [ ]:
import numpy as np
from layers.utils.LSTMCell import LSTMCell
from model.Model import Model
from utils.model_loss import cross_entropy_loss_npdl

In [ ]:
def create_toy_cell():
    np.random.seed(0)
    weights = {'Wc_i': np.random.rand(3, 5),
               'Wu_i': np.random.rand(3, 5),
               'Wf_i': np.random.rand(3, 5),
               'Wo_i': np.random.rand(3, 5),
               'Wc_h': np.random.rand(5, 5),
               'Wu_h': np.random.rand(5, 5),
               'Wf_h': np.random.rand(5, 5),
               'Wo_h': np.random.rand(5, 5)}
    biases = {'bc': np.zeros(5),
              'bu': np.zeros(5),
              'bf': np.zeros(5),
              'bo': np.zeros(5)}
    np.random.seed(0)
    return LSTMCell(3, 5, weights, biases)

def create_toy_data():
    np.random.seed(1)
    return np.random.uniform(-1, 1, (5, 3)), np.random.uniform(-1, 1, (5, 5)), np.random.uniform(-1, 1, (5, 5)), np.array([0, 1, 1, 4, 2])

In [ ]:
cell = create_toy_cell()
x, h, c, y = create_toy_data()

In [ ]:
correct_h_next = np.array([[ 0.05432073, -0.0240766, -0.08374854, 0.00076676, -0.14279187],
                           [-0.26776613, 0.31730641, -0.21700223, -0.28390231, -0.26996079],
                           [-0.13447024, -0.08660263, -0.10022437, -0.03162882, -0.05513277],
                           [-0.07544577, 0.17997456, -0.08875356, 0.28115015, 0.28349453],
                           [-0.20265419, -0.04393417, 0.15789945, -0.00593344, -0.18382524]])

correct_c_next = np.array([[ 0.14534732, -0.12841532, -0.17270371, 0.0028584, -0.33097237],
                           [-0.40553132, 0.52289938, -0.46666314, -0.41915372, -0.52302227],
                           [-0.5563115, -0.20751267, -0.33762134, -0.14975814, -0.34766355],
                           [-0.12754703, 0.35234149, -0.1480342, 0.517628, 0.47883466],
                           [-0.30773508, -0.14781391, 0.21382212, -0.01451843, -0.31231354]])

c_next, h_next, h_up  = cell.forward_npdl(x, h, c)

In [ ]:
print('Your h_next:')
print(h_next)
print()
print('correct h_next:')
print(correct_h_next)
print()

# La différence devrait être assez basse, en principe inférieure à 1e-7.
print('Difference between your h_next and correct h_next: ', np.sum(np.abs(h_next - correct_h_next)))

In [ ]:
print('Your c_next:')
print(c_next)
print()
print('correct c_next:')
print(correct_c_next)
print()

# La différence devrait être assez basse, en principe inférieure à 1e-7.
print('Difference between your c_next and correct c_next: ', np.sum(np.abs(c_next - correct_c_next)))

# TimeDistributed

In [ ]:
from layers.Dense import Dense
from layers.TimeDistributed import TimeDistributed
from utils.model_loss import td_cross_entropy_loss_npdl

In [ ]:
def create_toy_timedist():
    np.random.seed(0)
    return TimeDistributed(Dense(dim_input=5, dim_output=2), out_size=2)

def create_toy_sequence():
    np.random.seed(1)
    return np.random.uniform(-1, 1, (3, 4, 5)), np.array([[1, 1, 1, 0],
                                                          [1, 1, 0, 1],
                                                          [0, 1, 1, 1]])  

In [ ]:
layer = create_toy_timedist()
x, y = create_toy_sequence()

In [ ]:
z = layer.forward_npdl(x)
z

In [ ]:
loss, dScores, softmax_output = td_cross_entropy_loss_npdl(z, y, 0.0, {'l1': layer.get_params()})

In [ ]:
dX = layer.backward_npdl(dScores)
dX

# Sentiment analysis

In [ ]:
import numpy as np
import pandas as pd
from bpemb import BPEmb

In [ ]:
# Dataset: Stanford Sentiment Treebank V1.0
dictionary = pd.read_csv('datasets/stanfordSentimentTreebank/dictionary.txt', header=None, sep='|')
dictionary = dictionary.rename(columns={0:'phrase', 1:'phrase_id'})

dataset_split = pd.read_csv('datasets/stanfordSentimentTreebank/datasetSplit.txt', sep=',')

dataset_sentences = pd.read_csv('datasets/stanfordSentimentTreebank/datasetSentences.txt', sep='\t')

dataset_labels = pd.read_csv('datasets/stanfordSentimentTreebank/sentiment_labels.txt', sep='|')
dataset_labels = dataset_labels.rename(columns={'phrase ids':'phrase_id', 'sentiment values':'sentiment'})

In [ ]:
# Obtenir les phrase_id des sentence
sentences_merged = dataset_sentences.merge(dictionary, left_on='sentence', right_on='phrase', how='left').drop(columns=['phrase'])

# Retirer les sentence qui n'ont pas de phrase_id
sentences_clean = sentences_merged[~sentences_merged.phrase_id.isnull()]

In [ ]:
# obtenir les labels
sentences_with_labels = sentences_clean.merge(dataset_labels, on='phrase_id', how='left').drop(columns=['phrase_id'])

In [ ]:
# separation train - valid - test
sentences_split = sentences_with_labels.merge(dataset_split, on='sentence_index')

# Embeddings

In [ ]:
bpemb_en = BPEmb(lang="en", dim=25, vs=100000)

In [ ]:
def call_embed(value, embedder, max_length):
    emb = embedder.embed(value)
    return np.pad(emb, ((0, max_length - emb.shape[0]), (0, 0)), 'constant', constant_values=(0))

def get_longest(value, embedder):
    emb = embedder.embed(value)
    return emb.shape[0]

def convert_sentiment(value):
    if value <= 0.4:
        return 0
    if value <= 0.6:
        return 1
    return 2
    

sentences_split['len'] = sentences_split.apply(lambda x: get_longest(x['sentence'], bpemb_en), axis=1)

max_len = sentences_split.len.max()
print(max_len)

sentences_split['embedding'] = sentences_split.apply(lambda x: call_embed(x['sentence'], bpemb_en, max_len), axis=1)
sentences_split['sentiment_label'] = sentences_split.apply(lambda x: convert_sentiment(x['sentiment']), axis=1)

In [ ]:
train = sentences_split[sentences_split.splitset_label == 1].drop(columns=['splitset_label'])
valid = sentences_split[sentences_split.splitset_label == 2].drop(columns=['splitset_label'])
test = sentences_split[sentences_split.splitset_label == 3].drop(columns=['splitset_label'])

In [ ]:
train_data = np.array(train.embedding.tolist())
valid_data = np.array(valid.embedding.tolist())
test_data = np.array(test.embedding.tolist())

train_labels = np.array(train.sentiment_label.tolist())
valid_labels = np.array(valid.sentiment_label.tolist())
test_labels = np.array(test.sentiment_label.tolist())

train_data = np.concatenate((train_data, test_data), axis=0)
train_labels = np.concatenate((train_labels, test_labels), axis=0)

# Modèle LSTM

In [ ]:
from model.Model import Model
from layers.LSTM import LSTM
from layers.Dense import Dense
from layers.Flatten import Flatten
from utils.model_loss import cross_entropy_loss_npdl

def create_lstm_network():
    model = Model()
    
    lstm1 = LSTM(58, 25, 50, weight_scale=None)
    dense1 = Dense(50, 3, weight_scale=None)
    
    flatten = Flatten()
    
    model.add(lstm1)
    model.add(flatten)
    model.add(dense1)
    model.add_loss(cross_entropy_loss_npdl)
    return model


In [ ]:
model = create_lstm_network()
predictions = model.predict(test_data[:32])
print(predictions)

In [ ]:
scores = model.forward_npdl(train_data[:32])
loss, dScores, softmax_output = model.calculate_loss(scores, train_labels[:32], 0.0)
print(loss)

In [ ]:
from model.Solver import epoch_solver_npdl, Adam, SGD

model = create_lstm_network()

optimizer = Adam(1e-3, model)
    
loss_history, train_accuracy_history, val_accuracy_history = epoch_solver_npdl(train_data, 
                                                                          train_labels,
                                                                          valid_data,
                                                                          valid_labels,
                                                                          2e-4,
                                                                          optimizer,
                                                                          lr_decay=0.95,
                                                                          batch_size=16,
                                                                          epochs=10)